In [1]:
import pickle
# import hanlp
import matplotlib.pyplot as plt
import requests
import random
import json
import numpy as np
# from gensim.models.keyedvectors import KeyedVectors
from transformers import BertTokenizer, BertModel

In [2]:
# 余弦相似度
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

# 欧氏距离
def euc_dist(vector_a, vector_b):
    return np.sqrt(np.sum(np.square(vector_a-vector_b)))

In [3]:
# 翻译
def translate(sent):
    API_KEY = 'HZAnl45C1imoFTwpXLrupO3Z'
    Secret_Key = 'ojsKK6BAp1sIDd1OoeFIBhdWCpYYDFMM'
    host = f'https://aip.baidubce.com/oauth/2.0/token?grant_type' \
           f'=client_credentials&client_id={API_KEY}&client_secret={Secret_Key}'
    response = requests.get(host)
    if response:
        token = response.json()['access_token']
    else:
        raise AssertionError('no Baidu access token')
    url = 'https://aip.baidubce.com/rpc/2.0/mt/texttrans/v1?access_token=' + token

    term_ids = ''  # 术语库id，多个逗号隔开

    # Build request
    headers = {'Content-Type': 'application/json'}
    payload = {'q': sent, 'from': 'zh', 'to': 'en', 'termIds': term_ids}

    # Send request
    result = requests.post(url, params=payload, headers=headers).json()
    translation = result['result']['trans_result'][0]['dst']
    return translation

In [4]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd
import seaborn as sns

def plot(qa_embeddings, cls_embeddings):
    reduced_rep = TSNE(n_components=2).fit_transform(np.concatenate((qa_embeddings, cls_embeddings), axis=0))
    # reduced_rep = TSNE(n_components=2).fit_transform(np.concatenate((qa_embeddings, cls_embeddings), axis=0))
    dim1 = reduced_rep[:, 0].tolist()
    dim2 = reduced_rep[:, 1].tolist()
    # get DataFrame

    data = {'dim1': dim1, 'dim2': dim2, 'labels': [1, 1, 6, 6, 7, 0, 1, 2, 3, 4, 5, 6], 'method': 't-SNE'}
    data = pd.DataFrame(data)

    # plot
    g = sns.relplot(
        data=data, x="dim1", y="dim2",
        col='method', hue='labels',
        kind="scatter",
    )

In [5]:
# 视角平移
VIEW_SHIFT = [
    '向左转',
    '转到后面',
    '转向右边',
    '视角转到下面',
    '视角平移到上面',
]


In [6]:

print('============== Loading Model ==============')
# 下载语料库
# HanLP = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH)

# # GloVe Model
# word2vec_path = 'embedding/60000_glove.840B.300d.txt'
# w2v = load_w2v(word2vec_path)
#
# labels = pickle.load(open('embedding/label_embedding.pkl', 'rb'))

# Bert Model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')


============== Loading Model ==============


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
# 类别描述
cls_desc = {
    'View angle translation or pan up': 'Move or translate the position of the current view',
    'View angle rotation or turn': 'Rotate the current view',
    'Rotate the view around the selected object': 'Rotates the position of the camera with the selected Game '
                                                  'Object as the center and the up vector as the axis to '
                                                  'transform the current view , while keeping the camera view '
                                                  'towards the Game Object',
    'Object translation': 'Moves or translates the currently selected Game Object',
    'Object rotates around itself': 'Rotates the currently selected Game Object',
    'Object scaling or shrinking': 'Scales the currently selected Game Object',
    'Object initialization or drawing': 'Initialize or create a Object and places it in the center of the current view'
}
cls_desc

{'View angle translation or pan up': 'Move or translate the position of the current view',
 'View angle rotation or turn': 'Rotate the current view',
 'Rotate the view around the selected object': 'Rotates the position of the camera with the selected Game Object as the center and the up vector as the axis to transform the current view , while keeping the camera view towards the Game Object',
 'Object translation': 'Moves or translates the currently selected Game Object',
 'Object rotates around itself': 'Rotates the currently selected Game Object',
 'Object scaling or shrinking': 'Scales the currently selected Game Object',
 'Object initialization or drawing': 'Initialize or create a Object and places it in the center of the current view'}

In [41]:

print('============== Embedding ==============')
# BERT list(cls_desc.keys()) [f'{k}: {v}' for k, v in cls_desc.items()]
cls_encoder_inputs = tokenizer(list(cls_desc.keys()), padding=True, max_length=128, return_tensors='pt')
cls_src_ids = cls_encoder_inputs['input_ids']
cls_embeddings = bert(cls_src_ids).pooler_output.detach().numpy()


============== Embedding ==============


In [5]:

# 示例
zh_sentences = [
    '视角平移到上方10米',         # 0
    '向左转',              # 1
    '向左转三十度',           # 1
    '绕这个正方形旋转10度',         # 2
    '把这个球移到下边10米',         # 3
    '球绕自身转20度',         # 4
    '三角缩小3倍',         # 5
    '正方形放大5倍',         # 5
    '画一个球',             # 6
    '画一个正方体',          # 6
    '把球放到正方体上面',    # 7
]

labels = [0, 1, 1, 2, 3, 4, 5, 5, 6, 6, 7]
# 翻译
en_sentences = [translate(sent) for sent in zh_sentences]
en_sentences

['Shift the viewing angle to 10 meters above',
 'turn left',
 'Turn 30 degrees to the left',
 'Rotate 10 degrees around the square',
 'Move the ball 10 meters below',
 'The ball turns 20 degrees around itself',
 'The triangle shrinks three times',
 'The square is magnified five times',
 'Draw a ball',
 'Draw a cube',
 'Put the ball on the cube']

In [35]:
# Encode
encoder_inputs = tokenizer(en_sentences, padding=True, max_length=128, return_tensors='pt')
src_ids = encoder_inputs['input_ids']
qa_embeddings = bert(src_ids).pooler_output.detach().numpy()


In [42]:

print('============== Calculating ==============')
print(labels)
for id, embedding in enumerate(qa_embeddings):
    # HanLP(sentences).pretty_print()
    # HanLP(sentences, tasks='dep').pretty_print()    # 依存句法分析
    # HanLP(sentences, tasks='con').pretty_print()    # 成分句法分析
    # HanLP(sentences, tasks='sdp').pretty_print()    # 语义依存分析
    # doc = HanLP(sentences, tasks='srl')    # 语义角色标注

    # srl = hanlp.load('CPB3_SRL_ELECTRA_SMALL')

    # Zero-shot Spoken language understanding
    # embedding = embed(command, w2v)
    sim_scores = [euc_dist(embedding, v) for v in cls_embeddings]   # euc_dist cos_sim
    print(labels[id], end=' ')
    print(np.argmax(sim_scores), end=' ')
    print(sim_scores)

============== Calculating ==============
[0, 1, 1, 2, 3, 4, 5, 5, 6, 6, 7]
0 5 [10.787177, 6.649132, 8.298123, 9.303526, 7.676928, 10.8794, 7.1741505]
1 1 [5.062075, 8.245573, 7.35057, 5.1877546, 7.0155625, 3.5614161, 7.3387046]
1 0 [6.331967, 3.5217288, 5.0384927, 5.0072956, 4.0022964, 6.0577426, 3.124009]
2 5 [6.5677633, 3.1809664, 4.1198897, 5.256631, 3.8612318, 6.902634, 3.6209843]
3 5 [6.22111, 2.917345, 5.0363426, 5.224271, 3.440804, 6.4388785, 2.9890478]
4 0 [4.9859867, 3.1988487, 3.3530314, 3.638092, 2.3436596, 4.914801, 3.2194612]
5 5 [11.19339, 6.934324, 8.603414, 9.65838, 7.760285, 11.263309, 7.5425944]
5 0 [11.952486, 7.7927403, 9.09316, 10.329334, 8.679075, 11.938218, 8.399675]
6 1 [5.5840077, 6.141852, 5.4478927, 4.4723186, 5.2357507, 3.7815983, 5.439824]
6 1 [4.042338, 5.844912, 5.190799, 3.1062808, 4.8135347, 2.223173, 4.978388]
7 5 [11.539995, 7.4035954, 9.119689, 10.068905, 8.421448, 11.685634, 7.859868]


In [37]:
zh_sentences

['视角平移到上方10米',
 '向左转',
 '向左转三十度',
 '绕这个正方形旋转10度',
 '把这个球移到下边10米',
 '球绕自身转20度',
 '三角缩小3倍',
 '正方形放大5倍',
 '画一个球',
 '画一个正方体',
 '把球放到正方体上面']

In [38]:
en_sentences

['Pan up to 10 meters',
 'turn left',
 'Turn 30 degrees to the left',
 'Rotate 10 degrees around the square',
 'Move the ball 10 meters below',
 'The ball turns 20 degrees around itself',
 'The triangle shrinks three times',
 'The square is magnified five times',
 'Draw a ball',
 'Draw a cube',
 'Put the ball on the cube']

In [43]:
list(cls_desc.keys())

['View angle translation or pan up',
 'View angle rotation or turn',
 'Rotate the view around the selected object',
 'Object translation',
 'Object rotates around itself',
 'Object scaling or shrinking',
 'Object initialization or drawing']

In [14]:
# prep_sents = [
#         'There is a cube on the left of another cube.',
#         'There is a sphere under a cube.',
#         'There are some columns in front of a cube.',
#         'I want two cubes behind three desks.'
#     ]
prep_sents = ['There is a bird flying above the sea.',
 'There is a radio on the table.',
 'I see a boat on the river.',
 'There is a big diamond in the sky.',
 'There are two trees in the field.',
 'There are two trees in front of the flower.']

In [1]:
import hanlp
HanLP = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH)

In [15]:

doc = HanLP(prep_sents, tasks='pos')    # 依存句法分析
# doc = HanLP(prep_sents, tasks='dep')   # 依存句法分析
# HanLP(prep_sents, tasks='dep')

In [16]:
doc

{'tok/fine': [['There',
   'is',
   'a',
   'bird',
   'flying',
   'above',
   'the',
   'sea',
   '.'],
  ['There', 'is', 'a', 'radio', 'on', 'the', 'table', '.'],
  ['I', 'see', 'a', 'boat', 'on', 'the', 'river', '.'],
  ['There', 'is', 'a', 'big', 'diamond', 'in', 'the', 'sky', '.'],
  ['There', 'are', 'two', 'trees', 'in', 'the', 'field', '.'],
  ['There', 'are', 'two', 'trees', 'in', 'front', 'of', 'the', 'flower', '.']],
 'pos/ctb': [['PN', 'VC', 'CD', 'NN', 'NN', 'JJ', 'DT', 'NR', 'PU'],
  ['PN', 'VC', 'CD', 'NN', 'P', 'DT', 'NR', 'PU'],
  ['PN', 'VV', 'CD', 'VA', 'P', 'DT', 'NN', 'PU'],
  ['PN', 'VC', 'CD', 'JJ', 'NN', 'P', 'DT', 'NR', 'PU'],
  ['PN', 'VC', 'CD', 'NN', 'P', 'DT', 'NN', 'PU'],
  ['PN', 'VC', 'CD', 'NN', 'P', 'JJ', 'P', 'DT', 'NN', 'PU']]}

In [17]:
tokens = doc['tok/fine']
tags = doc['pos/ctb']

In [18]:
for i, sent in enumerate(prep_sents):
    token = tokens[i]
    tag = tags[i]
    print(sent, token, tag)
    if 'P' in tag:
        print(tag.index('P'))

There is a bird flying above the sea. ['There', 'is', 'a', 'bird', 'flying', 'above', 'the', 'sea', '.'] ['PN', 'VC', 'CD', 'NN', 'NN', 'JJ', 'DT', 'NR', 'PU']
There is a radio on the table. ['There', 'is', 'a', 'radio', 'on', 'the', 'table', '.'] ['PN', 'VC', 'CD', 'NN', 'P', 'DT', 'NR', 'PU']
4
I see a boat on the river. ['I', 'see', 'a', 'boat', 'on', 'the', 'river', '.'] ['PN', 'VV', 'CD', 'VA', 'P', 'DT', 'NN', 'PU']
4
There is a big diamond in the sky. ['There', 'is', 'a', 'big', 'diamond', 'in', 'the', 'sky', '.'] ['PN', 'VC', 'CD', 'JJ', 'NN', 'P', 'DT', 'NR', 'PU']
5
There are two trees in the field. ['There', 'are', 'two', 'trees', 'in', 'the', 'field', '.'] ['PN', 'VC', 'CD', 'NN', 'P', 'DT', 'NN', 'PU']
4
There are two trees in front of the flower. ['There', 'are', 'two', 'trees', 'in', 'front', 'of', 'the', 'flower', '.'] ['PN', 'VC', 'CD', 'NN', 'P', 'JJ', 'P', 'DT', 'NN', 'PU']
4


In [27]:
exp1_dep = doc['dep'][0]
exp1_sent = doc['tok/fine'][0]
print(exp1_sent)
print(exp1_dep)

['There', 'is', 'a', 'cube', 'on', 'the', 'left', 'of', 'another', 'cube', '.']
[(2, 'top'), (0, 'root'), (4, 'nummod'), (2, 'attr'), (2, 'prep'), (7, 'det'), (8, 'pobj'), (5, 'conj'), (10, 'amod'), (8, 'pobj'), (2, 'punct')]


In [24]:
for idx, trip in enumerate(exp1_dep):
    tp = trip[1]
    if tp == 'prep':
        print(tp)
        print(exp1_sent[idx])

prep
on


In [12]:

HanLP(prep_sents, tasks='con').pretty_print()    # 成分句法分析

In [14]:

HanLP(prep_sents, tasks='sdp').pretty_print()    # 语义依存分析

In [15]:

HanLP(prep_sents, tasks='srl')    # 语义角色标注

{'tok/fine': [['There',
   'is',
   'a',
   'cube',
   'on',
   'the',
   'left',
   'of',
   'another',
   'cube',
   '.'],
  ['There', 'is', 'a', 'sphere', 'under', 'a', 'cube', '.'],
  ['There', 'are', 'some', 'columns', 'in', 'front', 'of', 'a', 'cube', '.'],
  ['I', 'want', 'two', 'cubes', 'behind', 'three', 'desks', '.']],
 'srl': [[[('There', 'ARG0', 0, 1), ('is', 'PRED', 1, 2)]],
  [],
  [[('There', 'ARG0', 0, 1), ('are', 'PRED', 1, 2)]],
  [[('I', 'ARG0', 0, 1),
    ('want', 'PRED', 1, 2),
    ('twocubesbehindthreedesks', 'ARG1', 2, 7)]]]}